In [ ]:
import pandas as pd
import urllib.request
import shutil
import os
import pathlib
import csv
import traceback

%load_ext autoreload
%autoreload 2
from utils import *

In [ ]:
# Get a list of all videos
with open("links.csv", "r") as f:
    videos = [l.split(",")[0].strip() for l in f.readlines()]
    videos = list(filter(lambda x: len(x) > 1,videos))
    
videos += ["QYKPEz0J"] # Melissa, lab
print(videos)

In [ ]:
# Download keypoints of a given subject
def download_subject(slug):
    print(slug)
    target_path = "videos/keypoints/{}".format(slug)
    if os.path.exists(target_path):
        print("{} exists".format(target_path))
        return
    filepath = "/tmp/{}.tar.gz".format(slug)
    url = "https://mc-motionlab-storage.s3.us-west-2.amazonaws.com/media/outputs/{}/output.tar.gz".format(slug)
    print(url)
    urllib.request.urlretrieve(url, filepath)
    
    os.makedirs("/tmp/{}".format(slug), exist_ok=True)
    os.system("tar -zxvf {} -C /tmp/{}".format(filepath, slug))   
    
    shutil.move("/tmp/{}/output/keypoints".format(slug), target_path)

In [ ]:
# Download all subjects
for video in videos:
    try:
        download_subject(video)
    except:
        print("Error: "+video)
        traceback.print_exc()

In [ ]:
# Convert to numpy arrays and save locally
os.makedirs("videos/np", exist_ok=True)

for subjectid in videos:
    try:
        res = json2np("videos/keypoints/{}".format(subjectid),subjectid)
        np.save("videos/np/{}.npy".format(subjectid), res)
    except:
        print("error " + subjectid)